In [2]:
from sympy.solvers import solve
from sympy import Symbol, log, exp
import sympy as sp

In [3]:
def get_prior(target_palloff, num_features=500):
    x = Symbol('x')
    eq = sp.Eq(num_features*log(1-x), log(target_palloff))
    sol = solve(eq, x)
    assert len(sol) == 1
    return sol[0]

In [4]:
prior = get_prior(0.4)

In [5]:
print(prior)

0.00183090331161104


In [6]:
def sigmoid(x):
    return 1 / (1 + exp(-x))

def derive_params(prior_palloff, target_palloff, label=True, num_features=500):
    prior = get_prior(prior_palloff, num_features=num_features)
    target_posterior = get_prior(target_palloff, num_features=num_features)
        
    log_alpha = Symbol('log_alpha')
    # if label is True, subtract; else, add
    if label:
        term = prior_palloff*log_alpha
    else:
        term = -1*prior_palloff*log_alpha
    
    eq = sp.Eq(sigmoid(log(prior)-log(1-prior)+term), target_posterior)
    sol = solve(eq, log_alpha)
    
#     assert len(sol) == 1, sol
    sol = sol[0]
    
    lla = log(sol)
    
    return prior, lla

In [14]:
# get hyperparams for positive updates (unif, eig_train_model)
num_features = 500 # our estimate for num features in a random sequence
for prior_palloff in [0.5, 0.3, 0.1]:
    print("")
    print(f"prior palloff: {prior_palloff}")
    for posterior_palloff in [0.4, 0.3, 0.2, 0.1, 0.05, 0.01]:
        if posterior_palloff >= prior_palloff:
            continue
        
        prior, lla = derive_params(prior_palloff, posterior_palloff, label=True, num_features=num_features)

        print(f'prior={prior}\tlla={lla}\t\t(posterior_palloff={posterior_palloff})')


prior palloff: 0.5
prior=0.00138533389897108	lla=-0.582269524436040		(posterior_palloff=0.4)
prior=0.00138533389897108	lla=0.100117997493529		(posterior_palloff=0.3)
prior=0.00138533389897108	lla=0.522731931474560		(posterior_palloff=0.2)
prior=0.00138533389897108	lla=0.877263465295861		(posterior_palloff=0.1)
prior=0.00138533389897108	lla=1.07568861873668		(posterior_palloff=0.05)
prior=0.00138533389897108	lla=1.33374133882702		(posterior_palloff=0.01)

prior palloff: 0.3
prior=0.00240504883318384	lla=-0.0316148863778620		(posterior_palloff=0.2)
prior=0.00240504883318384	lla=0.772427968403266		(posterior_palloff=0.1)
prior=0.00240504883318384	lla=1.11334210926109		(posterior_palloff=0.05)
prior=0.00240504883318384	lla=1.50033511229104		(posterior_palloff=0.01)

prior palloff: 0.1
prior=0.00459458264847305	lla=0.970210628375177		(posterior_palloff=0.05)
prior=0.00459458264847305	lla=1.93939240693089		(posterior_palloff=0.01)


In [15]:
# get hyperparams for positive updates (unif, eig_train_model)
for prior_palloff in [0.5, 0.3, 0.1]:
    print("")
    print(f"prior palloff: {prior_palloff}")
    for posterior_palloff in [0.8, 0.9, 0.95, 0.99]:
        if posterior_palloff <= prior_palloff:
            continue
        
        prior, lla = derive_params(prior_palloff, posterior_palloff, label=False)

        print(f'prior={prior}\tlla={lla}\t\t(posterior_palloff={posterior_palloff})')


prior palloff: 0.5
prior=0.00138533389897108	lla=0.818807677680882		(posterior_palloff=0.8)
prior=0.00138533389897108	lla=1.32677908055553		(posterior_palloff=0.9)
prior=0.00138533389897108	lla=1.65032042172503		(posterior_palloff=0.95)
prior=0.00138533389897108	lla=2.13636980699140		(posterior_palloff=0.99)

prior palloff: 0.3
prior=0.00240504883318384	lla=1.72665652700838		(posterior_palloff=0.8)
prior=0.00240504883318384	lla=2.09477871455906		(posterior_palloff=0.9)
prior=0.00240504883318384	lla=2.35358707233326		(posterior_palloff=0.95)
prior=0.00240504883318384	lla=2.76987047901232		(posterior_palloff=0.99)

prior palloff: 0.1
prior=0.00459458264847305	lla=3.15104773856974		(posterior_palloff=0.8)
prior=0.00459458264847305	lla=3.42965456729417		(posterior_palloff=0.9)
prior=0.00459458264847305	lla=3.63928993623763		(posterior_palloff=0.95)
prior=0.00459458264847305	lla=3.99571598252335		(posterior_palloff=0.99)
